In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 588.3 MB 16 kB/s 
     |████████████████████████████████| 439 kB 47.0 MB/s 
     |████████████████████████████████| 1.7 MB 54.5 MB/s 
     |████████████████████████████████| 6.0 MB 46.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

Mounted at /content/drive


In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
17869,cheap chic like wanted funky hotel paying chic...,1
18562,beautiful resort firstly little bit travel yea...,1
12500,"n't stay, n't stay, punta cana 2002 wonderful ...",0
18482,"aewsome hotel lovely, clean tidy, great breakf...",1
11398,terrible loved infinity pool family took trip ...,0
8804,"faded glamour, forgive writing present tense h...",0
20258,great rooms golf corse beach lovely big apartm...,1
18660,"resort, absolutly beautiful resort hot humid, ...",1
16365,"ca n't meet obligations singaporean, reservati...",0
19429,great hotel excellent boutique hotel modern co...,1


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
927,curate egg accepting centrally located elderly...,0
1128,pleasant experience pleased october 8-10 stay ...,1
192,"parking fee stinks, husband booked room year h...",0
918,great room location service price recently sta...,1
348,unclean terrible service stayed save money 3 d...,0
747,great stay intercontinental great time big eas...,1
2253,standards gone downhill katrina stayed hotel p...,0
940,charming hotel lovely staff charming hotel art...,1
1908,great no hot water arrived late cold forever c...,0
4057,smokers delight joined husband business trip s...,0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
albert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")



In [ ]:
albert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = albert_preprocess(text_input)
outputs = albert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
49/49 [==============================] - 780s 16s/step - loss: 0.7568 - accuracy: 0.5215 - precision: 0.5068 - recall: 0.4443
Epoch 2/15
49/49 [==============================] - 757s 15s/step - loss: 0.6733 - accuracy: 0.6012 - precision: 0.5860 - recall: 0.6008
Epoch 3/15
49/49 [==============================] - 757s 15s/step - loss: 0.6381 - accuracy: 0.6307 - precision: 0.6234 - recall: 0.5995
Epoch 4/15
49/49 [==============================] - 749s 15s/step - loss: 0.6065 - accuracy: 0.6699 - precision: 0.6791 - recall: 0.6034
Epoch 5/15
49/49 [==============================] - 752s 15s/step - loss: 0.5876 - accuracy: 0.6904 - precision: 0.6878 - recall: 0.6605
Epoch 6/15
49/49 [==============================] - 750s 15s/step - loss: 0.5700 - accuracy: 0.7187 - precision: 0.7283 - recall: 0.6684
Epoch 7/15
49/49 [==============================] - 760s 16s/step - loss: 0.5519 - accuracy: 0.7354 - precision: 0.7552 - recall: 0.6711
Epoch 8/15
49/49 [=======================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted


146/146 [==============================] - 2284s 16s/step


array([[0.63357925],
       [0.14537986],
       [0.0529144 ],
       ...,
       [0.69904643],
       [0.2674161 ],
       [0.952958  ]], dtype=float32)

In [ ]:
import numpy as np


y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [ ]:
y_test







4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [46]:
y_test_array = np.array(y_test)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)

0.7578676942838792

In [48]:
from sklearn.metrics import precision_score
precision_score(y_test_array, y_predicted)



0.8396396396396396

In [49]:
from sklearn.metrics import recall_score
recall_score(y_test_array, y_predicted)

0.6180371352785146

In [50]:
from sklearn.metrics import f1_score
f1_score(y_test_array, y_predicted)

0.7119938884644766

In [51]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [52]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test_array, y_predicted)

In [53]:
# trained on hotels

In [54]:
df_results_for_reviews_on_hotels

,accuracy,precision,recall,f1_score
reviews on hotels,0.757868,0.83964,0.618037,0.711994


In [55]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/albert_trained_on_hotel_reviews_23_december_15_epochs'

In [56]:
print(saved_model_path)




/content/drive/MyDrive/data_for_colab/albert_trained_on_hotel_reviews_23_december_15_epochs


In [57]:
model.save(saved_model_path, include_optimizer=True) 






